In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
df = pd.read_csv("ChocolateSales.csv", sep=",")
# https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales



In [3]:
df.dtypes

Sales Person     object
Country          object
Product          object
Date             object
Amount           object
Boxes Shipped     int64
dtype: object

In [4]:
df = df.astype({
    "Sales Person": str,
    "Country":str,
    "Product": str,
    "Boxes Shipped": int
})

df['Amount_type'] = df['Amount'].apply(lambda x: type(x).__name__)
df['Amount'] =df['Amount'].str.replace(r'$', '')
df['Amount'] =df['Amount'].str.replace(r',', '')
df['Amount']= df['Amount'].astype(float)
df

,Sales Person,Country,Product,Date,Amount,Boxes Shipped,Amount_type
0,Jehu Rudeforth,UK,Mint Chip Choco,04-Jan-22,5320.0,180,str
1,Van Tuxwell,India,85% Dark Bars,01-Aug-22,7896.0,94,str
2,Gigi Bohling,India,Peanut Butter Cubes,07-Jul-22,4501.0,91,str
3,Jan Morforth,Australia,Peanut Butter Cubes,27-Apr-22,12726.0,342,str
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24-Feb-22,13685.0,184,str
...,...,...,...,...,...,...,...
1089,Karlen McCaffrey,Australia,Spicy Special Slims,17-May-22,4410.0,323,str
1090,Jehu Rudeforth,USA,White Choc,07-Jun-22,6559.0,119,str
1091,Ches Bonnell,Canada,Organic Choco Syrup,26-Jul-22,574.0,217,str
1092,Dotty Strutley,India,Eclairs,28-Jul-22,2086.0,384,str


In [5]:
df.isna().sum()

Sales Person     0
Country          0
Product          0
Date             0
Amount           0
Boxes Shipped    0
Amount_type      0
dtype: int64

In [6]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')
df['Year'] = df['Date'].dt.year

In [7]:
df.groupby('Country')['Amount'].sum()

Country
Australia      1137367.0
Canada          962899.0
India          1045800.0
New Zealand     950418.0
UK             1051792.0
USA            1035349.0
Name: Amount, dtype: float64

In [8]:
# df2 = pd.DataFrame(["Product", "Sum"])
df2 = df.groupby('Product').agg(Total=('Amount', 'sum')).reset_index()
top_sell_brands = df2.sort_values(by="Total", ascending=False).head(5)

In [11]:
pd.to_datetime(df['Date'], format='%b')
frame = pd.DataFrame({
    'date': pd.to_datetime(df['Date'], format='mixed', dayfirst=True) })
df['month'] = frame['date'].dt.month_name()
df

,Sales Person,Country,Product,Date,Amount,Boxes Shipped,Amount_type,Year,month
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320.0,180,str,2022,January
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896.0,94,str,2022,August
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501.0,91,str,2022,July
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726.0,342,str,2022,April
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685.0,184,str,2022,February
...,...,...,...,...,...,...,...,...,...
1089,Karlen McCaffrey,Australia,Spicy Special Slims,2022-05-17,4410.0,323,str,2022,May
1090,Jehu Rudeforth,USA,White Choc,2022-06-07,6559.0,119,str,2022,June
1091,Ches Bonnell,Canada,Organic Choco Syrup,2022-07-26,574.0,217,str,2022,July
1092,Dotty Strutley,India,Eclairs,2022-07-28,2086.0,384,str,2022,July


In [19]:
avg_total_per_month = df.groupby( 'month').agg(Total=('Amount', 'mean')).reset_index()
avg_total_per_month.sort_values('Total', ascending=False)

,month,Total
2,February,6357.972727
3,January,5818.863636
6,March,5721.244275
0,April,5712.296610
7,May,5576.977778
1,August,5545.880597
4,July,5392.114094
5,June,5307.631902


In [40]:
df_h = pd.DataFrame(df[['Product', 'Amount', 'Boxes Shipped']])
df_h['price_ind'] = round(df_h['Amount'] /df_h['Boxes Shipped'],2 )


,Product,Amount,Boxes Shipped,price_ind
0,Mint Chip Choco,5320.0,180,29.56
1,85% Dark Bars,7896.0,94,84.00
2,Peanut Butter Cubes,4501.0,91,49.46
3,Peanut Butter Cubes,12726.0,342,37.21
4,Peanut Butter Cubes,13685.0,184,74.38
...,...,...,...,...
1089,Spicy Special Slims,4410.0,323,13.65
1090,White Choc,6559.0,119,55.12
1091,Organic Choco Syrup,574.0,217,2.65
1092,Eclairs,2086.0,384,5.43


In [10]:
df.head(10)

,Sales Person,Country,Product,Date,Amount,Boxes Shipped,Amount_type,Year
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320.0,180,str,2022
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896.0,94,str,2022
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501.0,91,str,2022
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726.0,342,str,2022
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685.0,184,str,2022
5,Van Tuxwell,India,Smooth Sliky Salty,2022-06-06,5376.0,38,str,2022
6,Oby Sorrel,UK,99% Dark & Pure,2022-01-25,13685.0,176,str,2022
7,Gunar Cockshoot,Australia,After Nines,2022-03-24,3080.0,73,str,2022
8,Jehu Rudeforth,New Zealand,50% Dark Bites,2022-04-20,3990.0,59,str,2022
9,Brien Boise,Australia,99% Dark & Pure,2022-07-04,2835.0,102,str,2022
